# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


In [2]:
df= pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [ ]:
len(df)

In [10]:
df.group.value_counts()

control       4264
experiment    3924
Name: group, dtype: int64

In [13]:
df.group.value_counts().unique()

array([4264, 3924])

In [11]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [12]:
df.action.value_counts().unique()

array([6328, 1860])

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [26]:
#find mean of clicks for control (c) and experiment(e)
dc = len(df.loc[df['group']=='control'])
de = len(df.loc[df['group']=='experiment'])

nc = len(df.loc[(df['group']=='control') & (df['action']=='click')])
ne = len(df.loc[(df['group']=='experiment') & (df['action']=='click')])

In [31]:
print("Size of control group =", dc, "Num. of clicks =", nc)
print("Size of experiment group =", de, "Num. of clicks =", ne)

print("Average clicks for Control Group:", nc/dc)
print("Average clicks for Experiment Group:", ne/de)

Size of control group = 4264 Num. of clicks = 932
Size of experiment group = 3924 Num. of clicks = 928
Average clicks for Control Group: 0.21857410881801126
Average clicks for Experiment Group: 0.23649337410805302


In [28]:
control= df.loc[df['group']=='control']
control['clicks']= np.where(control['action']=='click', 1,0)


<ipython-input-28-924d98e35d0e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['clicks']= np.where(control['action']=='click', 1,0)


In [31]:
exp= df.loc[df['group']=='experiment']
exp['clicks']= np.where(exp['action']=='click', 1,0)

<ipython-input-31-547d3e916467>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exp['clicks']= np.where(exp['action']=='click', 1,0)


In [38]:
print("Control Mean:", control.clicks.mean())
print("Exp Mean:", exp.clicks.mean())

Control Mean: 0.21857410881801126
Exp Mean: 0.23649337410805302


In [41]:
#fs.p_value_welch_ttest(control.click, exp.click)
print(stats.ttest_ind(control.clicks, exp.clicks, equal_var = False))


Ttest_indResult(statistic=-1.9312441445072492, pvalue=0.05348777384439879)


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [44]:
cr=control.clicks.mean()
n=len(exp)
er=cr*n
er

857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [45]:
var= n*cr*(1-cr)
std=np.sqrt(var)
std

25.888551746650283

In [47]:
aclicks = exp.clicks.sum()
z = (aclicks - er)/std
z

2.71607302278784

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [49]:
p_val = 1 - stats.norm.cdf(z)
p_val

0.0033030672759265522

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.